<a href="https://colab.research.google.com/github/deconasser/UdemyCourse/blob/main/UnetPolypSegmantationv_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow
!pip install segmentation_models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install albumentations

In [38]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations import (
    HorizontalFlip, RandomRotate90, ShiftScaleRotate, ElasticTransform, GridDistortion, OpticalDistortion, Compose, OneOf
)
from albumentations import Compose, HorizontalFlip, RandomRotate90

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [5]:
# Đường dẫn tới dữ liệu
train_images_path = '/content/drive/MyDrive/Unet/train'
train_masks_path = '/content/drive/MyDrive/Unet/train_gt'

def load_data(image_path, mask_path, img_size=(256, 256)):
    images = []
    masks = []

    image_files = os.listdir(image_path)
    mask_files = os.listdir(mask_path)

    for img_file, mask_file in zip(image_files, mask_files):
        img = load_img(os.path.join(image_path, img_file), target_size=img_size)
        img = img_to_array(img)

        mask = load_img(os.path.join(mask_path, mask_file), color_mode="grayscale", target_size=img_size)
        mask = img_to_array(mask)

        images.append(img)
        masks.append(mask)

    images = np.array(images, dtype=np.float32) / 255.0
    masks = np.array(masks, dtype=np.float32) / 255.0

    return images, masks

In [19]:
train_images, train_masks = load_data(train_images_path, train_masks_path)

In [20]:
print(train_images.shape)
print(train_masks.shape)

(1000, 256, 256, 3)
(1000, 256, 256, 1)


In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(2, 3, activation='relu', padding='same')(conv9)

    segmentation_output = layers.Conv2D(1, 1, activation='sigmoid', name='segmentation')(conv9)

    model = models.Model(inputs=inputs, outputs=segmentation_output)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

model = unet_model()
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_92 (Conv2D)          (None, 256, 256, 64)         1792      ['input_5[0][0]']             
                                                                                                  
 conv2d_93 (Conv2D)          (None, 256, 256, 64)         36928     ['conv2d_92[0][0]']           
                                                                                                  
 max_pooling2d_16 (MaxPooli  (None, 128, 128, 64)         0         ['conv2d_93[0][0]']           
 ng2D)                                                                                      

In [22]:
model = unet_model()

In [23]:
# Backbone definition
BACKBONE = 'resnet34'

# Data augmentation using albumentations
def augment_data(image, mask):
  aug = Compose([
      HorizontalFlip(p=0.5),
      RandomRotate90(p=0.5),
      ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
      OneOf([
          ElasticTransform(p=0.5),
          GridDistortion(p=0.5),
          OpticalDistortion(p=0.5)
      ], p=0.5),
      A.RandomBrightnessContrast(p=0.5)
  ])
  augmented = aug(image=image, mask=mask)
  return augmented['image'], augmented['mask']

In [62]:
def data_generator(images, masks, batch_size):
    while True:
        for start in range(0, len(images), batch_size):
            end = min(start + batch_size, len(images))
            batch_images = []
            batch_masks = []
            for i in range(start, end):
                if i >= len(images) or i >= len(masks):
                    print(f"Index {i} is out of bounds for images or masks")
                    continue
                image, mask = augment_data(images[i], masks[i])
                batch_images.append(image)
                batch_masks.append(mask)
            yield np.array(batch_images), np.array(batch_masks)


In [53]:

print(len(train_images))

1000


In [61]:

# Normalize images
train_images_tmp, val_images, train_masks_tmp, val_masks = train_test_split(train_images, train_masks, test_size=0.2, random_state=42)
print("Total images:", len(train_images_tmp))
print("Total masks:", len(train_masks_tmp))
print("Total validation images:", len(val_images))
print("Total validation masks:", len(val_masks))
# Create generators
train_generator = data_generator(train_images_tmp, train_masks_tmp, batch_size=16)
print(train_generator)
val_generator = data_generator(val_images, val_masks, batch_size=16)

# Callbacks
callbacks = [
  EarlyStopping(patience=10, restore_best_weights=True),
  ModelCheckpoint('best_model.keras', save_best_only=True)
]


Total images: 800
Total masks: 800
Total validation images: 200
Total validation masks: 200
<generator object data_generator at 0x7bccf42c9460>


In [63]:

# Train the model
history = model.fit(
  train_generator,
  steps_per_epoch=len(train_images_tmp) // 16,
  epochs=50,
  validation_data=val_generator,
  validation_steps=len(val_images) // 16,
  callbacks=callbacks
)


Epoch 1/50
50/50 [==============================] - 56s 1s/step - loss: 0.1359 - accuracy: 0.9411 - val_loss: 0.0832 - val_accuracy: 0.9438
Epoch 2/50
50/50 [==============================] - 56s 1s/step - loss: 0.0879 - accuracy: 0.9411 - val_loss: 0.0793 - val_accuracy: 0.9443
Epoch 3/50
50/50 [==============================] - 53s 1s/step - loss: 0.0846 - accuracy: 0.9419 - val_loss: 0.0801 - val_accuracy: 0.9433
Epoch 4/50
50/50 [==============================] - 61s 1s/step - loss: 0.0828 - accuracy: 0.9422 - val_loss: 0.0774 - val_accuracy: 0.9443
Epoch 5/50
50/50 [==============================] - 54s 1s/step - loss: 0.0842 - accuracy: 0.9415 - val_loss: 0.0799 - val_accuracy: 0.9420
Epoch 6/50
50/50 [==============================] - 61s 1s/step - loss: 0.0835 - accuracy: 0.9413 - val_loss: 0.0804 - val_accuracy: 0.9421
Epoch 7/50
50/50 [==============================] - 60s 1s/step - loss: 0.0837 - accuracy: 0.9411 - val_loss: 0.0798 - val_accuracy: 0.9423
Epoch 8/50
50/50 [==

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import albumentations as A
import cv2
import matplotlib.pyplot as plt

# Tải mô hình đã lưu
model = load_model('best_model.keras', compile=False)

# Hàm tiền xử lý dữ liệu đầu vào
def preprocess_image(image):
    transform = A.Compose([
        A.Resize(256, 256),
        A.Normalize()
    ])
    return transform(image=image)['image']

# Hàm để hiển thị ảnh và mặt nạ
def display_image_and_mask(image, mask):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(mask, cmap="gray")
    ax[1].set_title('Predicted Mask')
    ax[1].axis('off')

    plt.show()


OSError: No file or directory found at best_model.keras

In [ ]:
# Ví dụ về dữ liệu đầu vào (thay thế bằng dữ liệu thực tế của bạn)
input_image = cv2.imread('/content/drive/MyDrive/Unet/test/02fa602bb3c7abacdbd7e6afd56ea7bc.jpeg')  # Thay thế bằng đường dẫn tới ảnh thực tế
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)  # Chuyển đổi từ BGR sang RGB nếu cần
original_size = input_image.shape[:2]  # Lưu kích thước gốc của ảnh

# Tiền xử lý ảnh đầu vào
preprocessed_image = preprocess_image(input_image)
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)  # Thêm batch dimension

# Thực hiện dự đoán
predictions = model.predict(preprocessed_image)
print(predictions)

predicted_mask = cv2.resize(predicted_mask.astype(np.uint8), (original_size[1], original_size[0]))  # Chuyển đổi kích thước về kích thước gốc

# Tạo mặt nạ nhị phân (màu trắng cho phần được dự đoán)
binary_mask = np.where(predicted_mask == 1, 255, 0).astype(np.uint8)
# Hiển thị ảnh gốc và mặt nạ dự đoán
display_image_and_mask(input_image, binary_mask)